# Setting up in the new network / for new accounts:
- This notebook allows to prepare wallets, tokens and contracts for running scenarios.
- You would need to update the consts in the [/scripts/defaults.py](../../scripts/defaults.py) to use deployed tokens and contracts in the scenarios.

In [1]:
from docs.scenarios.setup import *

web3, etherlink_account, tezos_account = setup()

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `299.499474 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `502.165002855 ꜩ`


### Deposit XTZ to L2:
- This call will fund default etherlink_account on Etherlink side by utilizing native deposit bridge.

In [2]:
opg_hash = xtz_deposit.callback(
    xtz_ticket_helper=XTZ_TICKET_HELPER,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    amount=1_000_000,
    receiver_address=etherlink_account.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
)

Making XTZ deposit using Helper `KT1VEjeQfDBSfpDH5WeBM5LukHPGM2htYEh3`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
  - XTZ deposit params:
      * Smart Rollup address: `sr18wx6ezkeRjt1SZSeZ2UQzQN3Uc3YLMLqg`
      * Receiver address: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
      * Amount (mutez): `1_000_000`
Successfully executed XTZ deposit, tx hash: `oo6QQmo9mb8PPPXzta9MPjJ2D2oCLSGxtKrN353EXGGXGvvKLtY`


### Deploy L1 token:
- This call will deploy new token, that can be used as a testing token in the scenarios.
- This token should be stored in the Ticketer contract storage to be used in other test scenarios.

In [4]:
token = deploy_token.callback(
    token_type='FA2',
    token_id=9000,
    total_supply=70_000_000,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    skip_confirm=True,
)

- You will need to update the `TEZOS_TOKEN_ADDRESS` and `TEZOS_TOKEN_TYPE` variables in the [/scripts/defaults.py](../../scripts/defaults.py), to use this token contract in the test scenarios.

In [5]:
click.echo("TEZOS_TOKEN_ADDRESS = '" + accent(token.address) + "'")
standard = token.make_token_info()['token_type']
click.echo("TEZOS_TOKEN_TYPE = '" + accent(standard) + "'")

TEZOS_TOKEN_ADDRESS = 'KT1GowwNbv8e6x4yRMazgFSqEQ63FySfjMVL'
TEZOS_TOKEN_TYPE = 'FA2'


### Deploy bridge:
- This call will deploy bridge contracts on both Tezos and Etherlink side for the provided token.
- You can both use existing token from the network or deploy you own test token.

In [6]:
tezos_bridge_contracts = bridge_token.callback(
    token_address=token.address,
    token_type='FA2',
    token_id=9000,
    token_decimals=0,
    token_symbol='vSecT',
    token_name='Various Security Testing Scenarios',
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
    kernel_address=KERNEL_ADDRESS,
    skip_confirm=True
)

ticketer = tezos_bridge_contracts['ticketer']
erc20 = tezos_bridge_contracts['erc20']
token_bridge_helper = tezos_bridge_contracts['token_bridge_helper']

Deploying bridge contracts for `vSecT`:
  - Token contract: `KT1GowwNbv8e6x4yRMazgFSqEQ63FySfjMVL`
  - Token id: `9000`

Deploying Ticketer for `vSecT`:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - Params:
      * Token type: `FA2`
      * Token address: `KT1GowwNbv8e6x4yRMazgFSqEQ63FySfjMVL`
      * Token id: `9000`
      * Token symbol: `vSecT`
      * Token name: `Various Security Testing Scenarios`
      * Token decimals: `0`
Successfully deployed Ticketer, address: `KT1TGqLzaj9q75TsRZgg7mCTGQxmHbi7bWxD`

Deploying ERC20 Proxy for `vSecT`:
  - Deployer: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://etherlink.dipdup.net`
  - Constructor params:
      * Ticketer address bytes: `0x01cd060ecba3492b343d42741a56f8ed55aab04f4500`
      * Content bytes: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b5431476f77774e6276386536783479524d617a67465371455

- You will need to update the `TICKETER_ADDRESS`, `ERC20_PROXY_ADDRESS`, `TOKEN_BRIDGE_HELPER_ADDRESS` variables in the [/scripts/defaults.py](../../scripts/defaults.py), to use these contracts in other scenarios.

In [7]:
click.echo("TICKETER_ADDRESS = '" + accent(ticketer.address) + "'")
click.echo("ERC20_PROXY_ADDRESS = '" + accent(erc20.address) + "'")
click.echo("TOKEN_BRIDGE_HELPER_ADDRESS = '" + accent(token_bridge_helper.address) + "'")

TICKETER_ADDRESS = 'KT1TGqLzaj9q75TsRZgg7mCTGQxmHbi7bWxD'
ERC20_PROXY_ADDRESS = '0x03E39FF2b379FBcd9284Ab457113D82fF4daBBF4'
TOKEN_BRIDGE_HELPER_ADDRESS = 'KT1DJctAKPZwWJn921pEWauPH3Kt8vaDgnYo'


### Deploy TicketRouterTester:
- This is the tester-contract allowing to make arbitrary calls to the rollup, ticketer, accept tickets with different entrypoints, redirect them, mint tickets.

In [3]:
ticket_router_tester = deploy_ticket_router_tester(
    tezos_account,
    TEZOS_RPC_URL,
)

Deploying TicketRouterTester:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
Successfully deployed TicketRouterTester, address: `KT1KKHDuPeZ4KptN591TZ9UiCeKhHpKqaE3Y`


- You will need to update the `TICKET_ROUTER_TESTER_ADDRESS` variable in the [/scripts/defaults.py](../../scripts/defaults.py), to use this contracts in other scenarios.

In [4]:
click.echo("TICKET_ROUTER_TESTER_ADDRESS = '" + accent(ticket_router_tester.address) + "'")

TICKET_ROUTER_TESTER_ADDRESS = 'KT1KKHDuPeZ4KptN591TZ9UiCeKhHpKqaE3Y'


### Wrap tokens to tickets:
- To use tickets, that represent FA token, directly from tezos account, they should be converted without sending to the rollup.
- There is a helper to do this:

In [5]:
# Using USDt ticketer for Ghostnet:
ticketer = Ticketer.from_address(tezos_account, TICKETER_ADDRESS)
ticketer.address

'KT1S6Nf9MnafAgSUWLKcsySPNFLUxxqSkQCw'

In [6]:
opg_hash = wrap_tokens_to_tickets(tezos_account, ticketer, 1_000_000)

Wrapping FA2 token, address: `KT1V2ak1MfNd3w4oyKD64ehYU7K4CrpUcDGR`, id: `0` tokens to tickets:
  - Token holder: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer: `KT1S6Nf9MnafAgSUWLKcsySPNFLUxxqSkQCw`
  - Amount: `1_000_000`
Successfully wrapped, tx hash: `onycELXKSdo4gbTMoNdsLNGR7ak4pNykSsuHK5rhJ2AVxwrMYJC`
